### Adversarial Text Example Experiment Runner Template
11/6/2017 - Basic pipeline to run adversarial text generation experiments.

### Dataset Preparation
Base dataset: The Enron Spam Dataset: http://www2.aueb.gr/users/ion/data/enron-spam/ 
    

In [13]:
import os
import numpy as np
import torch
from collections import defaultdict, Counter

In [11]:
## Spam Preprocessing - UNIX Command line
# 1. Removed all \n and replaced with spaces: find . -type f -exec perl -i. -pe 's/\r?\n/ /' {} +
# 2. Concatenated all spam into a single file and all ham into a single file.
#      To concatenate within dirs: awk 1 enron1/ham/*.txt > enron1_ham.txt 
# 3. Randomly shuffled: shuf input > output
# 4. Create 80, 10, 10 train, val, and test splits.

# Total ham: 16545 messages; train/val/test = 13236, 1654, 1655
# Total spam: 17171 messages; train/val/test = 13736, 1717, 1718

1718

In [33]:

base_data_dir = "/cvgl2/u/catwong/cs332_final_project/data/"
classes = ['spam', 'ham']
vocabulary_filename = 'email_train_vocab.txt' 
# Truncation and vocabulary shortening:
# Using the train data only:
# 1. Truncate both the spam and ham messages to truncation_len characters (adding padding where needed).
# 2. From the truncated messages, compile a vocabulary of the class_vocabulary_size most frequent tokens for each class.
# 3. Write a vocabulary file composed of the full, combined vocabulary (ie. the most frequent tokens across both classes.)
truncation_len = 100
class_vocabulary_size = 3000

combined_vocab = []
for class_name in classes:
    filename = os.path.join(base_data_dir, 'train', class_name + '.txt') 
    print "Now processing: %s" % filename
    with open(filename) as f:
        all_lines = [line.strip().lower().split() for line in f.readlines()]
        
    # Truncate the files.
    truncated_lines = [line[:truncation_len] for line in all_lines]
    
    # Add tokens to the counter
    token_counts = Counter()
    for line in truncated_lines:
        token_counts.update(line)
    combined_vocab += [elem for (elem, count) in token_counts.most_common(class_vocabulary_size)]
    # Convert the combined vocabulary into a set.
combined_vocab = set(combined_vocab)
# Write out the combined_vocab to the vocabulary file
with open(vocabulary_filename, 'w') as f:
    for token in combined_vocab:
        f.write(token + "\n")

Now processing: /cvgl2/u/catwong/cs332_final_project/data/train/spam.txt
Now processing: /cvgl2/u/catwong/cs332_final_project/data/train/ham.txt


In [2]:
# Class that takes in a file and a vocabulary file (which has a truncation len) and converts the text into
# encoded/truncated sentences.

class DatasetEncoderDecoder(object):
    """
    Encodes and decodes sentences according to a vocabulary.
    
    Sentences are truncated. OOV words are assigned an <UNK> token, and <SOS>, <PAD>, and <EOS> tokens are added.
    
    truncation_len
    """
    def __init__(self, truncation_len, vocab_file):
        self.truncation_len = truncation_len
        # Create index to word and word to index dicts from the vocab_file.
        self.index2word = {0:'<SOS>', 1:'<EOS>', 2: '<UNK>', 3: '<PAD>'}
        self.word2index = {'<SOS>':0, '<EOS>':1, '<UNK>':}
        
    
    def encode_sentence(self, sentence):
        """
        Encodes a sentence according to the vocabulary.
        Returns:
            normalized: the normalized sentence, as it would be decoded.
            encoded: the encoded numerical sentence.
        """
        pass
    def decode_sentence(self, encoded):
        "

### Autoencoder - Seq2Seq Model
Source: http://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#the-seq2seq-model 